In [ ]:
!pip install "pyiceberg[s3fs,hive]"

In [ ]:
! pip install kafka-python

In [3]:
from pyiceberg.catalog import load_catalog
catalog = load_catalog(
        "docs",
    **{
            "uri": "thrift://hive-ms-trino.apps.zagaopenshift.zagaopensource.com:9888",
            "s3.endpoint": "http://minio-lb.apps.zagaopenshift.zagaopensource.com:9009",
            "py-io-impl": "pyiceberg.io.pyarrow.PyArrowFileIO",
            "s3.access-key-id": "minioAdmin",
            "s3.secret-access-key": "minio1234",
    }
)
logstable=catalog.load_table("docs_logs.logs") #mention tablename and namesapce

In [5]:
from kafka import KafkaConsumer
import json
try:
        # Create KafkaConsumer
        consumer = KafkaConsumer(
            "apmlogs",
            bootstrap_servers=['telemetry-kafka-external-bootstrap-observability-kafka.apps.zagaopenshift.zagaopensource.com:443'],
            value_deserializer=lambda v: v,  # Keep value as bytes for JSON decoding
            group_id="apmingestion",
            auto_offset_reset='earliest',
            consumer_timeout_ms=10000,
            security_protocol='SSL',  # Add SSL configuration if needed
            ssl_cafile='./telemetry_zagaopenshift.pem',
            ssl_certfile='./telemetry_zagaopenshift.crt'
        )
        print("Kafka consumer created successfully.")
except Exception as e:
        print(f"Error creating Kafka consumer: {e}")

Kafka consumer created successfully.


In [ ]:
import pyarrow as pa
import datetime
import json
def append_data(data):
  final={}
  # for key in data['resourceLogs'][0]['scopeLogs'] :
  #   for key0 in key['logRecords'] :
  #     # print(key0)
  #     final.update({
  #         "traceid":key0['traceId'],
  #     })
  for key in data['resourceLogs'][0]['resource']['attributes'] :
    if(key['key']=='service.name'):
      final.update({
          "servicename":key['value']['stringValue'],
      })
    if(key['key']=='telemetry.sdk.language'):
      final.update({
          "language":key['value']['stringValue'],
          "logsdata":json.dumps(data),
          "createdTime":datetime.datetime.now()
      })
      
      df=pa.Table.from_pylist([final])
      logstable.append(df)
      print("Data appended successfully.")
      # format.append(final)
      final={}
messages=[]
try:
  for message in consumer:
      data = json.loads(message.value.decode('utf-8'))
      append_data(data)
      # print(data)
      # messages.append(data)
      # if(len(messages) > 5 ):
      #   consumer.close()
except Exception as e:
  print(f"Error consuming messages: {e}")
finally:
  consumer.close()
  print("Message consumption closed.")

